# Transform data


Transforms the datasets we have to .xyz format, so they are compatible with the CryinGAN repository.

In [ ]:
import os
import ase
from ase.io import read, write
import numpy as np

from pathlib import Path

%cd ..

from src.utils import load_raw_data, read_raw_sample

%cd -

def makedir_if_not_exists(path):
    try:
        if not os.path.isdir(path):
            print("Creating directory {}".format(path))
            os.mkdir(path)
    except OSError:
        print("Creation of the directory %s failed" % path)
        makedir_if_not_exists(path.parent) # Recursive call to create parent directory
    return

path = Path("../data/raw/crystal/Sq")
path = Path("../data/raw/samples")


phis = [0.84]
files, dataframe, metadata = load_raw_data(path, phi=phis, subpath="")
files, dataframe, metadata = load_raw_data(path, phi=phis, subpath="")


In [ ]:
dataframe

In [ ]:
import pandas as pd

max_files = np.inf # NOTE: Limit the amount of data to speed up training

processed_folder = Path("../data/processed/crystal/Sq/disorder-0.1") # Output folder
processed_folder = Path("../data/processed/samples") # Output folder

makedir_if_not_exists(processed_folder)

# Dump as xyz with all data in a single file
processed_file = Path(files[0].replace("raw", "processed")).parent

makedir_if_not_exists(processed_file.parent)


if os.path.exists(processed_file):
    os.remove(processed_file)


for i, file in enumerate(files):
    dataframe, metadata = read_raw_sample(file)

    xyz = dataframe[["class", "x", "y"]].reset_index(drop=True)

    N = metadata.iloc[0,0] # N particles

    xyz["z"] = 0

    # Create an ASE Atoms object

    L = metadata["L"].iloc[0]

    # NOTE: Radius is not right
    atoms = ase.Atoms(
        numbers=xyz["class"].values,
        positions=xyz[["x", "y", "z"]].values+[L/2, L/2, 0], # NOTE: Displace the system
        cell=[L, L, 0], # NOTE: 2D system
        pbc=[True, True, False], # NOTE: 2D system
    )

    # Save to xyz file
    # NOTE: This is the format used by ASE

    with open(processed_file, "a+") as f:
        write(f, atoms, format="extxyz", append=True)

    # DEPRECATED: Save to xyz file by csv
    # xyz.to_csv(
    #     processed_file,
    #     header=False,
    #     index=False, 
    #     sep="\t", 
    #     mode="a+",
    #     )
    
    if i > max_files:
        break


display(dataframe.head(10))
metadata

In [ ]:
slab_from_file = read(processed_file, index=0, format="extxyz")

In [ ]:
from ase.visualize import view
view(slab_from_file)

Works